In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/FOREX/GU_merged.csv')

In [ ]:
df_1 = df_1.iloc[:,1:6]

In [ ]:
df_2 = pd.read_csv('/content/drive/MyDrive/FOREX/GBP_USD_202105.csv')

In [ ]:
df_2.drop('Volume', axis = 1, inplace=True)

In [ ]:
df_3 = pd.read_csv('/content/drive/MyDrive/FOREX/GBP_USD_202106.csv')

In [ ]:
df_3.drop('Volume', axis = 1, inplace=True)

In [ ]:
df_new = pd.concat([df_1,df_2,df_3], axis=0)

In [ ]:
df_new.drop_duplicates(inplace=True)

In [ ]:
# resample into H1
#  change timestamp to datetime object and set  as index
df_new['TimeStamp'] = pd.to_datetime(df_new['TimeStamp'])
df_new.set_index('TimeStamp', inplace=True, drop=True)
df_new_H1 = df_new.resample('H').agg({'Open':'first', 'High': np.max, 'Low': np.min, 'Close': 'last'})
df_new_B = df_new_H1.resample('B').agg({'Open':'first', 'High': np.max, 'Low': np.min, 'Close': 'last'})

In [ ]:
df_new.reset_index(inplace=True)

In [ ]:
df_new['Close_p4'] = df_new['Close'].shift(-3)
df_new['PIPS_H4'] = (df_new['Close_p4']-df_new['Open']) * 10000

In [ ]:
df_new.dropna(inplace=True)

In [ ]:
# to get dir without using loops - 0/0 will return nas
df_new['H4_Dir'] = df_new['PIPS_H4'] / abs(df_new['PIPS_H4'])

In [ ]:
df_new['H4_Dir'] = df_new['H4_Dir'].fillna(0)

In [ ]:
df_new.reset_index(inplace=True, drop=True)

In [ ]:
df_new.head()

,TimeStamp,Open,High,Low,Close,Close_p4,PIPS_H4,H4_Dir
0,2016-01-03 17:00:00,1.47335,1.47335,1.47329,1.47329,1.47332,-0.3,-1.0
1,2016-01-03 17:01:00,1.47328,1.47336,1.47326,1.47335,1.47332,0.4,1.0
2,2016-01-03 17:02:00,1.47335,1.47335,1.47329,1.47329,1.47332,-0.3,-1.0
3,2016-01-03 17:03:00,1.47330,1.47333,1.47329,1.47332,1.47331,0.1,1.0
4,2016-01-03 17:04:00,1.47332,1.47334,1.47332,1.47332,1.47331,-0.1,-1.0


In [ ]:
!pip install ta


In [ ]:
!pip install -U git+https://github.com/twopirllc/pandas-ta

  Cloning https://github.com/twopirllc/pandas-ta to /tmp/pip-req-build-m0ajuy43
  Running command git clone -q https://github.com/twopirllc/pandas-ta /tmp/pip-req-build-m0ajuy43
  Created wheel for pandas-ta: filename=pandas_ta-0.3.2b0-cp37-none-any.whl size=211144 sha256=1637e7114c5911d103beb6d29ea78830132466d654eda897de252de79e0d87b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-fbj5pd9w/wheels/64/67/96/15e918c3b53b4a323b5bd037c7f08be5ef6908141c50f07c76
Successfully built pandas-ta
  Found existing installation: pandas-ta 0.3.2b0
    Uninstalling pandas-ta-0.3.2b0:
      Successfully uninstalled pandas-ta-0.3.2b0


In [ ]:

from ta.trend import PSARIndicator 
from ta.momentum import RSIIndicator
from ta.momentum import StochasticOscillator
from ta.momentum import WilliamsRIndicator
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
from ta.trend import IchimokuIndicator
from ta.trend import MACD
#import pandas_ta

#Heikin-Ashi - renames cols

'''
df_new.ta.ha(append=True)
df_new.ta.cdl_doji(append=True)
df_new.ta.cdl_inside(append=True)
'''

#PSAR
psar = PSARIndicator(df_new['High'], df_new['Low'], df_new['Close'])
df_new['PSAR'] = psar.psar()
#PSAR indicators
df_new['PSAR_d_i'] =psar.psar_down_indicator()
df_new['PSAR_u_i'] =psar.psar_up_indicator()
#RSI 5 & 14
rsi_5 = RSIIndicator(df_new['Close'], 5)
rsi_14 = RSIIndicator(df_new['Close'], 14)

df_new['RSI_5'] = rsi_5.rsi()
df_new['RSI_14'] = rsi_14.rsi()

# stochastic oscillator
SO = StochasticOscillator(df_new['High'], df_new['Low'], df_new['Close'])
df_new['SO'] = SO.stoch()
df_new['SO_signal'] = SO.stoch_signal()

#william indicator
WI = WilliamsRIndicator(df_new['High'], df_new['Low'], df_new['Close'])
df_new['WI'] = WI.williams_r()

# Bollinger
Bol =  BollingerBands(df_new['Close'])
df_new['h_Bol'] = Bol.bollinger_hband()
df_new['m_Bol'] = Bol.bollinger_mavg()
df_new['l_Bol'] = Bol.bollinger_lband()

# check ta adx vs calculated
adx_ta = ADXIndicator(df_new['High'], df_new['Low'], df_new['Close'], 10)
df_new['ADX'] = adx_ta.adx()

# DI pos & neg
df_new['DI_pos'] = adx_ta.adx_pos() 
df_new['DI_neg'] = adx_ta.adx_neg() 
 

# check av true range ta vs calculated
atr = AverageTrueRange(df_new['High'], df_new['Low'], df_new['Close'])
df_new['ATR'] = atr.average_true_range() 

# sample standard deviation (n = 20), .std method defaults to n-1 in denominator
df_new['SSD'] = df_new['Close'].rolling(20).std()

# price moving averages

df_new['avg_price_5'] = df_new['Close'].rolling(window=5).mean()
df_new['avg_price_10'] = df_new['Close'].rolling(window=10).mean()

#Ichimoko
Ichimoku = IchimokuIndicator(df_new['High'], df_new['Low'],7,22,44) 
# Senkou Span A
df_new['Senkou_Span_A'] = Ichimoku.ichimoku_a()
# Senkou Span B
df_new['Senkou_Span_B'] = Ichimoku.ichimoku_b()
# Kijun sen
df_new['Kijun_sen'] = Ichimoku.ichimoku_base_line() 
# Tenkan sen
df_new['Tenkan-sen'] = Ichimoku.ichimoku_conversion_line() 

# MACD indicators
# MACD - defaults periods 26 & 12 and 9 for signal
macd = MACD(df_new['Close'])
df_new['MACD'] = macd.macd()
#DIF
df_new['DIF'] = macd.macd_diff() 
#DEA(signal)
df_new['DEA'] = macd.macd_signal()



/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [ ]:
df_new.isnull().sum()

TimeStamp         0
Open              0
High              0
Low               0
Close             0
Close_p4          0
PIPS_H4           0
H4_Dir            0
PSAR              0
PSAR_d_i          0
PSAR_u_i          0
RSI_5             4
RSI_14           13
SO               17
SO_signal        25
WI               17
h_Bol            19
m_Bol            19
l_Bol            19
ADX               0
DI_pos            0
DI_neg            0
ATR               0
SSD              19
avg_price_5       4
avg_price_10      9
Senkou_Span_A    21
Senkou_Span_B     0
Kijun_sen        21
Tenkan-sen        6
MACD             25
DIF              33
DEA              33
dtype: int64

In [ ]:
df_new.dropna(inplace=True)

In [ ]:
df_new['H4_Dir'] = df_new['H4_Dir'].astype('int')

In [ ]:
df_new.tail()

,TimeStamp,Open,High,Low,Close,Close_p4,PIPS_H4,H4_Dir,PSAR,PSAR_d_i,PSAR_u_i,RSI_5,RSI_14,SO,SO_signal,WI,h_Bol,m_Bol,l_Bol,ADX,DI_pos,DI_neg,ATR,SSD,avg_price_5,avg_price_10,Senkou_Span_A,Senkou_Span_B,Kijun_sen,Tenkan-sen,MACD,DIF,DEA
1697043,2021-06-11 16:51:00,1.41127,1.41127,1.41124,1.41125,1.41131,0.4,1,1.411058,0.0,0.0,51.173711,49.812394,35.384615,42.051282,-64.615385,1.411405,1.411214,1.411023,48.102893,44.384422,23.175040,0.000092,0.000098,1.411290,1.411207,1.411345,1.411410,1.411345,1.411345,-0.000016,0.000022,-0.000037
1697044,2021-06-11 16:52:00,1.41125,1.41127,1.41123,1.41124,1.41124,-0.1,-1,1.411070,0.0,0.0,49.650187,49.142842,33.846154,35.897436,-66.153846,1.411397,1.411210,1.411024,46.220045,42.501559,23.252458,0.000088,0.000096,1.411318,1.411212,1.411345,1.411400,1.411345,1.411345,-0.000014,0.000018,-0.000032
1697045,2021-06-11 16:53:00,1.41125,1.41128,1.41124,1.41126,1.41087,-3.8,-1,1.411082,0.0,0.0,53.138075,50.573773,36.923077,35.384615,-63.076923,1.411397,1.411210,1.411024,44.649998,41.713752,22.205783,0.000085,0.000096,1.411276,1.411220,1.411360,1.411400,1.411345,1.411375,-0.000011,0.000017,-0.000028
1697046,2021-06-11 16:54:00,1.41125,1.41133,1.41124,1.41131,1.41073,-5.2,-1,1.411094,0.0,0.0,61.477389,54.054235,44.615385,38.461538,-55.384615,1.411402,1.411212,1.411023,43.855987,43.113863,19.959648,0.000085,0.000097,1.411266,1.411239,1.411365,1.411360,1.411345,1.411385,-0.000005,0.000019,-0.000023
1697047,2021-06-11 16:55:00,1.41129,1.41135,1.41124,1.41124,1.41126,-0.3,-1,1.411106,0.0,0.0,46.878444,48.866224,33.846154,38.461538,-66.153846,1.411399,1.411211,1.411023,43.382444,40.102693,17.549020,0.000087,0.000097,1.411260,1.411256,1.411353,1.411345,1.411345,1.411360,-0.000005,0.000015,-0.000020


In [ ]:
#take 10% of data for faster training
training_set = df_new.iloc[int(-0.1*len(df_new)):,]

In [ ]:
training_set.head()

,TimeStamp,Open,High,Low,Close,Close_p4,PIPS_H4,H4_Dir,PSAR,PSAR_d_i,PSAR_u_i,RSI_5,RSI_14,SO,SO_signal,WI,h_Bol,m_Bol,l_Bol,ADX,DI_pos,DI_neg,ATR,SSD,avg_price_5,avg_price_10,Senkou_Span_A,Senkou_Span_B,Kijun_sen,Tenkan-sen,MACD,DIF,DEA
1527341,2020-12-27 22:31:00,1.35651,1.35652,1.35642,1.35650,1.35659,0.8,1,1.355867,0.0,0.0,80.905291,63.033130,90.361446,91.967871,-9.638554,1.356581,1.356182,1.355783,26.713761,35.359968,16.918881,0.000160,0.000204,1.356352,1.356153,1.356212,1.35615,1.356165,1.356260,0.000063,0.000045,0.000018
1527342,2020-12-27 22:32:00,1.35651,1.35656,1.35641,1.35652,1.35679,2.8,1,1.355910,0.0,0.0,81.940275,63.632792,92.771084,91.566265,-7.228916,1.356618,1.356194,1.355771,27.896901,34.538627,15.320389,0.000159,0.000217,1.356448,1.356225,1.356220,1.35615,1.356165,1.356275,0.000079,0.000049,0.000030
1527343,2020-12-27 22:33:00,1.35652,1.35665,1.35648,1.35664,1.35678,2.6,1,1.355950,0.0,0.0,87.159993,67.083034,98.888889,94.007140,-1.111111,1.356679,1.356213,1.355748,29.651005,36.495271,13.691455,0.000160,0.000239,1.356540,1.356294,1.356255,1.35615,1.356200,1.356310,0.000101,0.000057,0.000044
1527344,2020-12-27 22:34:00,1.35664,1.35665,1.35658,1.35659,1.35665,0.1,1,1.356006,0.0,0.0,75.756119,64.343857,93.333333,94.997769,-6.666667,1.356712,1.356223,1.355735,31.229699,34.802304,13.056327,0.000154,0.000251,1.356552,1.356358,1.356270,1.35615,1.356200,1.356340,0.000113,0.000055,0.000058
1527345,2020-12-27 22:35:00,1.35658,1.35686,1.35657,1.35679,1.35676,1.8,1,1.356058,0.0,0.0,85.343974,69.677428,93.693694,95.305305,-6.306306,1.356797,1.356249,1.355702,33.982909,41.420466,10.758920,0.000163,0.000281,1.356608,1.356434,1.356400,1.35617,1.356305,1.356495,0.000137,0.000063,0.000074


In [ ]:
training_set = training_set[['Open','High','Low','Close','ADX','avg_price_10','MACD','DIF','ATR','DEA','SSD','Senkou_Span_A','Senkou_Span_B','Kijun_sen','Tenkan-sen','H4_Dir']]

In [ ]:
train_X =training_set.iloc[:,0:15]
train_y = training_set.iloc[:,15]
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
sc = sc.fit(train_X)
X_sc = sc.fit_transform(train_X)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
# convert list to np array
trainY = np.array(train_y)

In [ ]:
#create model instances
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
#mnb & cnb not working with -ve target values
mnb = MultinomialNB(alpha=0.2)
cnb = CategoricalNB()
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)
abc = AdaBoostClassifier(n_estimators=62, random_state=111)
bc = BaggingClassifier(n_estimators=9, random_state=111)
etc = ExtraTreesClassifier(n_estimators=9, random_state=111)
dnn = MLPClassifier(solver='sgd', alpha=1e-5, learning_rate = 'adaptive', activation = 'logistic')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, trainY, test_size=0.2)

In [ ]:
# create variables with classifiers instances
clfs = {'SVC' : svc,'KN' : knc,'DT': dtc, 'LR': lrc, 'RF': rfc, 'AdaBoost': abc, 'BgC': bc, 'ETC': etc, 'DNN':dnn}

In [ ]:
def train_classifier(clf, feature_train, labels_train):    
    clf.fit(feature_train, labels_train)

In [ ]:
# function to predict using classifiers
def predict_labels(clf, features):
    return (clf.predict(features))

In [ ]:
pred_scores = []
for k,v in clfs.items():
    train_classifier(v, X_train, y_train)
    pred = predict_labels(v,X_test)
    print(k,classification_report(y_test,pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVC               precision    recall  f1-score   support

          -1       0.50      0.51      0.50     16639
           0       0.00      0.00      0.00       779
           1       0.50      0.51      0.51     16522

    accuracy                           0.50     33940
   macro avg       0.33      0.34      0.34     33940
weighted avg       0.49      0.50      0.49     33940



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KN               precision    recall  f1-score   support

          -1       0.54      0.57      0.56     16639
           0       0.00      0.00      0.00       779
           1       0.54      0.55      0.55     16522

    accuracy                           0.54     33940
   macro avg       0.36      0.37      0.37     33940
weighted avg       0.53      0.54      0.54     33940

DT               precision    recall  f1-score   support

          -1       0.56      0.58      0.57     16639
           0       0.06      0.05      0.05       779
           1       0.57      0.55      0.56     16522

    accuracy                           0.56     33940
   macro avg       0.40      0.39      0.40     33940
weighted avg       0.55      0.56      0.55     33940



/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LR               precision    recall  f1-score   support

          -1       0.63      0.63      0.63     16639
           0       0.00      0.00      0.00       779
           1       0.63      0.66      0.64     16522

    accuracy                           0.63     33940
   macro avg       0.42      0.43      0.42     33940
weighted avg       0.62      0.63      0.62     33940

RF               precision    recall  f1-score   support

          -1       0.70      0.72      0.71     16639
           0       0.25      0.03      0.06       779
           1       0.70      0.71      0.71     16522

    accuracy                           0.70     33940
   macro avg       0.55      0.49      0.49     33940
weighted avg       0.69      0.70      0.69     33940

AdaBoost               precision    recall  f1-score   support

          -1       0.51      0.55      0.53     16639
           0       0.00      0.00      0.00       779
           1       0.51      0.50      0.51     16522

    a

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
trainY

array([ 0,  1, -1, ..., -1, -1, -1])

In [ ]:
'''
train_size = int(len(trainX) * 0.9)
test_size = len(trainX) - train_size

X_train, X_test = trainX[0:train_size], trainX[train_size:len(trainX)]
y_train, y_test = trainY[0:train_size], trainY[train_size:len(trainY)]

'''

'\ntrain_size = int(len(trainX) * 0.9)\ntest_size = len(trainX) - train_size\n\nX_train, X_test = trainX[0:train_size], trainX[train_size:len(trainX)]\ny_train, y_test = trainY[0:train_size], trainY[train_size:len(trainY)]\n\n'

In [ ]:
# import Keras modules
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
model = Sequential()
model.add(LSTM(25, dropout=0.2, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(trainX, trainY, epochs=20, batch_size=50,  validation_split =0.2)


Epoch 1/20
2715/2715 [==============================] - 53s 13ms/step - loss: 0.7082 - accuracy: 0.4882 - val_loss: 0.6934 - val_accuracy: 0.4813
Epoch 2/20
2715/2715 [==============================] - 32s 12ms/step - loss: 0.6938 - accuracy: 0.4881 - val_loss: 0.6938 - val_accuracy: 0.4813
Epoch 3/20
2715/2715 [==============================] - 32s 12ms/step - loss: 0.6938 - accuracy: 0.4916 - val_loss: 0.6987 - val_accuracy: 0.4862
Epoch 4/20
2715/2715 [==============================] - 33s 12ms/step - loss: 0.6938 - accuracy: 0.4905 - val_loss: 0.6926 - val_accuracy: 0.4862
Epoch 5/20
2715/2715 [==============================] - 33s 12ms/step - loss: 0.6935 - accuracy: 0.4893 - val_loss: 0.6926 - val_accuracy: 0.4862
Epoch 6/20
2715/2715 [==============================] - 33s 12ms/step - loss: 0.6934 - accuracy: 0.4906 - val_loss: 0.6923 - val_accuracy: 0.4864
Epoch 7/20
2715/2715 [==============================] - 33s 12ms/step - loss: 0.6935 - accuracy: 0.4882 - val_loss: 0.6923 -

In [ ]:
kfold = KFold(n_splits = 5)

In [ ]:
estimator = KerasClassifier(build_fn = baseline_model, epochs = 20, batch_size = 10, verbose = 0)

In [ ]:
results = cross_val_score(estimator, trainX, trainY, cv = kfold)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
IndexError: tuple index out of range

  FitFailedWarning)
